In [2]:
from langchain_community.document_loaders import PyPDFLoader

#Load the data
loader = PyPDFLoader("Rapids_AI_Text_data.pdf")
data = loader.load()

In [3]:
data

[Document(metadata={'source': 'Rapids_AI_Text_data.pdf', 'page': 0}, page_content='HOME PAGE  \nRapids AI Rapidly Accesible for everyone  \nComputer Vision  \nObject Detection  \nOur object detection solutions utilize advanced algorithms to identify and locate objects in images \nor videos. With applications across industries, our technology enhances efficiency and enables \nautomated object recognition and tracking. From object detect ion and recognition to image \nclassification and segmentation, our solutions extract valuable insights from visual data.  \nOur robust object detection solutions deliver precise identification and localization for diverse \nvisual data. We help businesses unlock the full potential of visual data and accelerate their digital \ntransformation.  \n \nMachine Learning  \nNatural Language Processing  \nOur ML solutions encompass a wide range of applications, including natural language processing \n(NLP). By using advanced algorithms and data analysis, we dev

In [4]:
len(data)

30

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  53


In [7]:
docs[1]

Document(metadata={'source': 'Rapids_AI_Text_data.pdf', 'page': 0}, page_content='Our ML solutions encompass a wide range of applications, including natural language processing \n(NLP). By using advanced algorithms and data analysis, we develop intelligent systems to interpret \ntextual data, enabling tasks such as sentiment analysis, langu age translation, and chatbot \ndevelopment.  \nFrom sentiment analysis to text classification, our NLP solutions enhance decision -making and \nenable intelligent data processing. We empower organizations to gain a deeper understanding of \ntheir data, improve decision -making, and deliver enhanced user expe riences.  \nMachine Learning  \nBioinformatics  \nMachine learning in bioinformatics is revolutionizing the field of life sciences by enabling advanced \nanalysis of biological data. Our expertise lies in developing ML models and algorithms tailored for \nbioinformatics applications, such as genomic data anal ysis, protein structure prediction, 

In [8]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv() 

#Create Embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("This is RapidsAI Chat-Bot")
vector[:5]

C:\Users\The One\anaconda3\envs\env_langchian\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[0.06442474573850632,
 -0.07610521465539932,
 -0.013498559594154358,
 -0.017090842127799988,
 0.03803476318717003]

In [10]:
#Store in Chroma DB
vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [11]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke("which type of services provide RapidsAI")

In [12]:
len(retrieved_docs)

10

In [16]:
print(retrieved_docs[4])

page_content='Our Featured Services  
With a deep understanding of industry -specific challenges, RapidsAI provides tailored 
software solutions that address your unique needs. Whether it’s building scalable e -
commerce platforms, implementing enterprise resource planning systems, or creating 
data-driven analytics dashboards, we are committed to delivering exceptional results.  
Web Design  
Design Research  
Mobile Design  
Design Concepts  
UI/UX  
3D Visualization  
3D Animation  
Photogrammetry  
Projects  
RapidsAI > Projects  
17 
Story Character Generation  
18 
Censorship of Profanities  
Translation -Whisper  
Multilingual Audio -Visual Translation  
BrandDevice  
Brand and Product Detection  
Portfolio_Upwork (1)  
Smart Driving Assistant  
logo  
Logo Detection  
NudityDet_Portfolio_Upwork (3)' metadata={'page': 22, 'source': 'Rapids_AI_Text_data.pdf'}


In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.3, max_tokens=500)

In [18]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [19]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [22]:
response = rag_chain.invoke({"input": "what are the client reviews about rapidsai"})
print(response["answer"])

This document does not contain client reviews for RapidsAI. Therefore, I cannot answer your question. This document provides information about the services offered by RapidsAI. 

